generate a csv: date, stock_price, stock_momentum, stock_volatility, bond_price, bond_momentum, bond_volatility, correlation

In [42]:
# Download daily stock prices for IWDA.L and USAG.L from Yahoo Finance
import yfinance as yf
import pandas as pd
import numpy as np

# Define tickers and date range
symbol_stock = 'IWDA.L'
symbol_bond = 'CSBGU0.SW'
start_date = '2011-01-01'
end_date = pd.Timestamp.today().strftime('%Y-%m-%d')

# Download data
prices_stock = yf.download(symbol_stock, start=start_date, end=end_date, auto_adjust=True)
prices_bond = yf.download(symbol_bond, start=start_date, end=end_date, auto_adjust=True)

# Extract opening prices
prices_stock = prices_stock['Open']
prices_stock = prices_stock.rename(columns={symbol_stock: "price"})


# Extract opening prices
prices_bond = prices_bond['Open']
prices_bond = prices_bond.rename(columns={symbol_bond: "price"})

prices_stock.head()

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

[*********************100%***********************]  1 of 1 completed


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

[*********************100%***********************]  1 of 1 completed


Ticker,price
Date,
2011-01-04,28.490000
2011-01-05,28.490000
2011-01-06,28.490000
2011-01-07,28.490000
2011-01-10,28.309999


In [49]:
# Clean data using pandas functions, calculate momentum, volatility, and correlation
# Fill missing or zero values with previous day's price using pandas
prices_stock = prices_stock.replace(0, np.nan).ffill()
prices_bond = prices_bond.replace(0, np.nan).ffill()

# Calculate 20-day momentum (price change over 20 days)
prices_stock['momentum'] = prices_stock['price'].pct_change(periods=20)
prices_bond['momentum'] = prices_bond['price'].pct_change(periods=20)
prices_stock['momentum'] = prices_stock['momentum'].fillna(0)
prices_bond['momentum'] = prices_bond['momentum'].fillna(0)

# Calculate 20-day volatility (rolling std of daily returns)
prices_stock['volatility'] = prices_stock['price'].pct_change().rolling(window=20).std()
prices_bond['volatility'] = prices_bond['price'].pct_change().rolling(window=20).std()
prices_stock['volatility'] = prices_stock['volatility'].fillna(0)
prices_bond['volatility'] = prices_bond['volatility'].fillna(0)


In [ ]:
# Combine stock and bond prices by date, calculate rolling 20-day correlation
# Reset index to ensure 'date' is a column
prices_stock_df = prices_stock.reset_index().rename(columns={'Date':'date', 'price': 'stock_price', 'momentum' : 'stock_momentum', 'volatility': 'stock_volatility'})
prices_bond_df = prices_bond.reset_index().rename(columns={'Date':'date', 'price': 'bond_price', 'momentum' : 'bond_momentum', 'volatility': 'bond_volatility'})

prices_stock_df['date'] = pd.to_datetime(prices_stock_df['date']).dt.strftime('%Y-%m-%d')
prices_bond_df['date'] = pd.to_datetime(prices_bond_df['date']).dt.strftime('%Y-%m-%d')

# Merge on 'date'
merged = pd.merge(prices_stock_df, prices_bond_df, on='date', how='inner')

# Calculate rolling 20-day correlation between stock and bond prices
merged['correlation'] = merged['stock_price'].rolling(window=20).corr(merged['bond_price'])

merged['correlation'] = merged['correlation'].fillna(0)

# Save to CSV
merged.to_csv('dataset.csv', index=False)

# Show first 25 rows
merged.head(25)

Ticker,date,stock_price,stock_momentum,stock_volatility,bond_price,bond_momentum,bond_volatility,correlation
0,2011-01-04,28.490000,0.000000,0.000000,110.010002,0.000000,0.000000,0.000000
1,2011-01-05,28.490000,0.000000,0.000000,108.373802,0.000000,0.000000,0.000000
2,2011-01-06,28.490000,0.000000,0.000000,108.440002,0.000000,0.000000,0.000000
3,2011-01-07,28.490000,0.000000,0.000000,109.731201,0.000000,0.000000,0.000000
4,2011-01-10,28.309999,0.000000,0.000000,110.220001,0.000000,0.000000,0.000000
5,2011-01-11,28.680000,0.000000,0.000000,109.654800,0.000000,0.000000,0.000000
6,2011-01-12,28.730000,0.000000,0.000000,109.555496,0.000000,0.000000,0.000000
7,2011-01-13,29.090000,0.000000,0.000000,110.063103,0.000000,0.000000,0.000000
8,2011-01-14,29.219999,0.000000,0.000000,109.799698,0.000000,0.000000,0.000000
9,2011-01-17,29.160000,0.000000,0.000000,109.940002,0.000000,0.000000,0.000000
